In [1]:
%load_ext autoreload
%autoreload 2

<img src="https://blog.langchain.dev/content/images/2023/04/Screenshot-2023-04-20-at-4.49.13-PM.png" alt="A simple retrieval Q&A system" width="1000" height="400" class="alignnone size-full wp-image-100" >

In [25]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.document_loaders import PDFMinerLoader, TextLoader
from langchain.vectorstores import FAISS

# Helper function for printing docs

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

documents = PDFMinerLoader(file_path="./arxiv_papers/2203.02155.pdf").load()
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = text_splitter.split_documents(documents=documents)


Created a chunk of size 1385, which is longer than the specified 300
Created a chunk of size 491, which is longer than the specified 300
Created a chunk of size 486, which is longer than the specified 300
Created a chunk of size 480, which is longer than the specified 300
Created a chunk of size 671, which is longer than the specified 300
Created a chunk of size 1438, which is longer than the specified 300
Created a chunk of size 506, which is longer than the specified 300
Created a chunk of size 759, which is longer than the specified 300
Created a chunk of size 615, which is longer than the specified 300
Created a chunk of size 449, which is longer than the specified 300
Created a chunk of size 471, which is longer than the specified 300
Created a chunk of size 495, which is longer than the specified 300
Created a chunk of size 715, which is longer than the specified 300
Created a chunk of size 754, which is longer than the specified 300
Created a chunk of size 752, which is longer t

In [26]:
retriever = FAISS.from_documents(documents=texts, 
                                embedding=HuggingFaceEmbeddings()).as_retriever()

docs = retriever.get_relevant_documents("What is Instruction Fine Tuning?")
pretty_print_docs(docs)

Document 1:

Baselines. We compare the performance of our PPO models to our SFT models and GPT-3. We also
compare to GPT-3 when it is provided a few-shot preﬁx to ‘prompt’ it into an instruction-following
mode (GPT-3-prompted). This preﬁx is prepended to the user-speciﬁed instruction.6
----------------------------------------------------------------------------------------------------
Document 2:

QA prompt
instruction
QA + instruct

QA prompt
instruction
QA + instruct

0.750
0.774
0.760

0.448
0.419
0.420

0.228
0.211
0.250

0.312
0.340
0.335

0.193
0.212
0.218

0.721
0.753
0.773

0.430
0.413
0.419

0.229
0.232
0.261

0.220
0.414
0.348

0.186
0.212
0.267

0.735
0.796
0.783

0.410
0.362
0.353
----------------------------------------------------------------------------------------------------
Document 3:

Figure 5: Comparing our models with FLAN and T0 in terms of Likert scores on a 1-7 scale, on the
InstructGPT prompt distribution. FLAN and T0 perform better than default GPT-3, and com

In [ ]:
from langchain.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever


splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0, separator=". ")
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)


In [ ]:
compression_retriever = ContextualCompressionRetriever(base_compressor=pipeline_compressor, base_retriever=retriever)
compressed_docs = compression_retriever.get_relevant_documents("What did the president say about Ketanji Jackson Brown")
pretty_print_docs(compressed_docs)

In [19]:
#TODO: Add ContextualCompressionRetriever for compressing documents and querying them

from langchain.llms import HuggingFacePipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# local_llm = HuggingFacePipeline(pipeline=pipe)
pipe = pipeline("text2text-generation", model="google/flan-t5-large", max_length=1000)
llm = HuggingFacePipeline(pipeline=pipe)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

compressed_docs = compression_retriever.get_relevant_documents("What is Instruction Fine Tuning?")
pretty_print_docs(compressed_docs)

Document 1:

Mixing in pretraining updates performs better than the simpler solution of increasing the KL co- efficient. In Figure 33, we show that there is a value of the pretraining mix coefficient that both reverses the performance regressions on SQuADv2 and DROP (the datasets we used for testing), and has minimal reductions in validation reward.
----------------------------------------------------------------------------------------------------
Document 2:

Model samples on a prompt cherry-picked to show ability to follow instructions to describe code, along with random samples from the GPT-3 175B and InstructGPT 175B models.
----------------------------------------------------------------------------------------------------
Document 3:

InstructGPT generalizes ‘following instructions’ to settings that we don’t supervise it in, for example on non-English language tasks and code-related tasks
-------------------------------------------------------------------------------------------